In [1]:
import gymnasium as gym
import random
import numpy as np
import math
import pygame

In [2]:
env = gym.make("CliffWalking-v0",render_mode="human")

In [3]:
print(env.observation_space)
print(env.action_space)
action_space = [0,1,2,3]

Discrete(48)
Discrete(4)


In [4]:
# #Random testing
# episodes = 1
# for episode in range(0,episodes):
#     state = np.array(env.reset()[0])
#     done = False
#     score = 0
#     steps = 0
#     while not done and steps < 100:
#         action = random.choice(action_space)
#         observation, reward, done,_,_  = env.step(action)
#         steps+=1
#         score+=reward
#         env.render()
    
#     print(f"Episode {episode+1} : score = {score}")
# env.close()

In [5]:
StateGrid = [[i*12 + j for j in range(12)]for i in range(4)]
print(StateGrid)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]]


In [6]:
discount = 0.7

In [7]:
def ingrid(i,j):
    if i < 0 or j < 0 or j >= 12 or i >= 4:
        return False
    return True

def next_states(state):
    i = state//12
    j = state%12
    new_state = []
    #up
    if ingrid(i-1,j): new_state.append(StateGrid[i-1][j])
    else : new_state.append(state) 
    #right
    if ingrid(i,j+1): new_state.append(StateGrid[i][j+1])
    else : new_state.append(state) 
    #down
    if ingrid(i+1,j): new_state.append(StateGrid[i+1][j])
    else : new_state.append(state) 
    #left
    if ingrid(i,j-1): new_state.append(StateGrid[i][j-1])
    else : new_state.append(state) 
   
    return new_state

In [8]:
def reward_model(state):
    if(state == 47):
        return 1.0
    elif state <= 36:
        return -1.0
    elif 37 <= state <= 46:
        return -100.0
    else: 
        return -1.0
    

# policy Evaluation iteration
def policy_evaluation(values,policy):
    convergence = 0
    new_values = [-1*math.inf]*48
    for row in StateGrid:
        for state in row:
            nextStates = next_states(state)
            new_values[state] = reward_model(state) + discount*(values[nextStates[policy[state]]])
            convergence = max(convergence,abs(new_values[state]-values[state]))

    return (new_values , convergence)


# policy improvement iteration
def policy_improvement(values,policy):
    convergence = 1
    new_values = [-1*math.inf]*48
    for row in StateGrid:
        for state in row:
            nextStates = next_states(state)
            new_values[state] = reward_model(state) + discount*(values[nextStates[policy[state]]])
            for i in range(4):
                if i != policy[state] and (reward_model(state) + discount*(values[nextStates[i]])) > new_values[state]:
                    new_values[state] = reward_model(state) + discount*(values[nextStates[i]])
                    policy[state] = i
                    convergence = 0
                    #print(f"updating to {state} from {policy[state]} to {i}")

    return (policy , convergence)
   


In [9]:
policy = random.choices([0,1,2,3],weights=[0.25,0.25,0.25,0.25],k=48)
convergence = 0


while not convergence:
    state_values = [0]*48
    delta = 1
    it = 0
    while delta > 0.01 and 10000 > it:
        state_values , delta = policy_evaluation(state_values,policy)
        it += 1

    policy , convergence = policy_improvement(state_values,policy)


print(policy)



[1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]


In [10]:
def predict_action(state):
    val = float(-1*math.inf)
    action_state = -1
    for states in next_states(state):
        if(state == -1): continue
        if val < state_values[states]:
            val = state_values[states]
            action_state = states 

    #left
    if action_state + 1 == state:
        return 3
    #down
    elif action_state - 12 == state:
        return 2
    #right
    elif action_state - 1 == state:
        return 1
    
    return 0

#Random testing
episodes = 1
for episode in range(0,episodes):
    state = np.array(env.reset()[0])
    done = False
    score = 0
    steps = 0
    while not done and steps < 100:
        action = policy[state]
        state, reward, done,_,_  = env.step(action)
        steps+=1
        score+=reward
        env.render()
    
    print(f"Episode {episode+1} : score = {score}")
env.reset()
pygame.quit()
pygame.display.quit()
env.close()



Episode 1 : score = -13


: 